# Working on Dataset by building KNN Classifier

Import Dataset from UCI Repository

In [23]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn import preprocessing
from datetime import datetime

Merge the features and targets into a single Dataset and preprocess it by normalization

In [24]:
#KNN
#Start here

X = load_breast_cancer()['data']
y = load_breast_cancer()['target']
names = load_breast_cancer()['feature_names']


full_df = pd.DataFrame(X, columns = names)
full_df['target'] = y
dtTemp = full_df.iloc[:,:-1]
x = full_df.iloc[:,:-1].values
min_max_scaler = preprocessing.MinMaxScaler(feature_range = (0,3))
x_scaled = min_max_scaler.fit_transform(x)
NormalizedDT = pd.DataFrame(x_scaled,columns=dtTemp.columns)
full_df = NormalizedDT
full_df['target'] = y

KNN Classifier which gives back a prediction list

In [25]:
def KNN_Classifier(dataFrame, unknown, K, min = 0, max = 1, p=2):
    import pandas as pd
    import numpy as np
    def Minkosky_Distance(dataFrame, unknown, p, k, min_normalization = 0, max_normalization = 1):
        features = dataFrame.columns[:-1]
        unknown_df = pd.DataFrame(unknown.reshape(1,len(features)), columns = features)
        df2 = pd.DataFrame()
        
        for feature in features:
            attr_max = dataFrame.describe()[feature][-1]
            attr_min = dataFrame.describe()[feature][3]
            df2[feature] = dataFrame[feature].map(lambda x: ((x-attr_min)*(max_normalization-min_normalization)/(attr_max-attr_min))-min_normalization)
            z = unknown_df[feature]
            unknown_df[feature] = ((z-attr_min)*(max_normalization-min_normalization)/(attr_max-attr_min))-min_normalization
        df2['target'] = dataFrame['target']
        X = df2.iloc[:,:-1]
        data = map(lambda row: list(map(lambda x,y: (abs(x-y))**p, row, unknown)),X.values)

        new_df = pd.DataFrame(data, columns = features)
        result = new_df.sum(axis=1).map(lambda total: total**(1/p)).sort_values(ascending=True)[:K]
        return result
    
    
    result = Minkosky_Distance(dataFrame, unknown, p, K, min_normalization = min, max_normalization = max)
    
    return dataFrame.iloc[list(result.index),:]['target'].value_counts().index[0]
    

def KNN_Multiple(dataFrame, test_data, K=3, min = 0, max = 1, p=2):
    import pandas as pd
    import numpy as np
    prediction = []
    for i in range(len(test_data)):
        prediction.append(KNN_Classifier(dataFrame, np.array(test_data.iloc[i]), K=3, min = 0, max = 1, p=2))

Splitting the dataset and also defining CV functions

In [26]:
def Train_Test_Split(dataFrame, k_fold = 10):
    
    n = math.ceil(len(dataFrame)/k_fold)
    
    train_test_list = []
        
    for i in range(k_fold):
        temp_tuple = (pd.concat([dataFrame[:(i*n)],dataFrame[((i+1)*n):]]),dataFrame[(i*n):((i+1)*n)])
        train_test_list.append(temp_tuple)
        
    return train_test_list
        
def Classifier(train_set,test_set, prediction):
    import pandas as pd
    import numpy as np
    X_train = train_set.iloc[:,:-1]
    y_train = train_set.iloc[:,-1]
    
    X_test = test_set.iloc[:,:-1]
    y_test = np.array(test_set.iloc[:,-1])  
    
    correct = 0
    for i in range(len(y_test)):
        if y_test[i] == prediction[i]:
            correct += 1
    return correct / float(len(y_test))

def KFold_CV(real,predicted,dataFrame):
    import pandas as pd
    import numpy as np
    total = len(dataFrame)
    correct = 0
    for i in range(len(real)):
        for j in range(len(real[i])):
            if (real[i][j] == predicted[i][j]):
                correct += 1

Calling the defined Cross Validation Functions

In [27]:
from KNN import KNN_Classifier, KNN_Multiple
from CrossValidation import Train_Test_Split, Classifier, KFold_CV 

Splitting the data into Testing and training sets

In [28]:
split_data = Train_Test_Split(full_df)

Create a prediction list with Timestamps

In [29]:
y_pred_list = []

for fold in split_data:

    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print("date and time =", dt_string)
    
    y_pred = KNN_Multiple(fold[0],fold[1].iloc[:,:-1])
    y_pred_list.append(y_pred)
    
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print("date and time =", dt_string)

date and time = 30/11/2019 17:56:41
date and time = 30/11/2019 18:12:14
date and time = 30/11/2019 18:12:19
date and time = 30/11/2019 18:27:22
date and time = 30/11/2019 18:27:22
date and time = 30/11/2019 18:42:49
date and time = 30/11/2019 18:42:50
date and time = 30/11/2019 18:58:00
date and time = 30/11/2019 18:58:00
date and time = 30/11/2019 19:12:57


Create a testing target list

In [30]:
y_test_list = []
for fold in split_data:
    y_test = list(fold[1].iloc[:,-1].values)
    y_test_list.append(y_test)

Calling the function which compares and returns the accuracy

In [31]:
KFold_CV(y_test_list,y_pred_list,full_df)

96.13356766256591

# THANK YOU